In [1]:
import numpy as np
import configurationsMulticlass
import funPytorch as fun
import pickle
import os

In [2]:
device = "cuda:0"
keepWords = [1,2,3,4,5,6,7,8,9,10,20,30,40,50,100]
corpusDir = "corpusSintex/mod-{}/keep-{}"
corpusFile = "corpusSintex.p"

conf = {
    'max': configurationsMulticlass.configTmulti1best,
    'soft': configurationsMulticlass.configTSmulti1best,
}

In [3]:
corpusKey = conf['max'].corpusKey

In [4]:
X, y, train, valid, test, allText = fun.processData(conf['max'], getText=True)

         processed line 85170/85170           
         reprocessed line 85170/85170           


In [5]:
model = {m:fun.loadModel(conf[m], device)[0] for m in conf}

In [6]:
values = pickle.load(open(conf['max'].fileValues, 'rb'))

In [7]:
dataset = {
    'train':train,
    'valid':valid,
    'test':test,
}

In [8]:
att ={m:{d:fun.getAttention(conf[m], model[m], X, y, dataset[d])[1] for d in dataset} for m in model}

In [9]:
for m in model:
    for d in dataset:
        if np.min(att[m][d]) < 0.:
            att[m][d] = att[m][d]/2 + 0.5

In [10]:
att['max']['train'].shape

(51101, 100, 512)

In [11]:
att['soft']['train'].shape

(51101, 100)

In [12]:
for d in dataset:
    att['max'][d] = np.sum(att['max'][d], axis=2)/512.

In [13]:
att['max']['train'].shape

(51101, 100)

In [14]:
i = {d:[values[currY] for currY in y[dataset[d]]] for d in dataset}
text = {d:[allText[c] for c in dataset[d]] for d in dataset}

In [15]:
for m in model:
    for k in keepWords:
        os.makedirs(corpusDir.format(m,k))
        currWordIndices = {d:np.argsort(att[m][d])[:,-k:] for d in dataset}
        currCorpus = {}
        for d in dataset:
            currCorpus[d] = {
                'text': np.array([' '.join([text[d][i][j] for j in currWordIndices[d][i] if j<len(text[d][i])]) for i in range(len(text[d]))]),
                corpusKey: np.array(i[d]),
            }
        pickle.dump(currCorpus, open(os.path.join(corpusDir.format(m,k), corpusFile), 'wb'))